We rarely get our data in just the form we want.  In this worksheet we will investigate some techniques for making relatively simple adjustments that help us get _from our input data into a data frame that is ready for us to use_.

We have some data in the file `iris.csv` which has been hand-transcribed from notebooks and contains a few errors.  To clean the data we will:
  * Load up the data
  * Understand the Data
  * Remove empty values
  * Fix format errors
  * Fix incorrect data
  * Remove duplicates

After this, we are ready to visualise the data (which we will learn more about next week.

In [2]:
import pandas as pd

# load data - notice that iris data has no index, so we will use a fresh one
iris = pd.read_csv("data/iris.csv")
iris


,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


Jupyter always gives us a preview of our data when we print it, but we can specifially ask for the `head` rows, the `tail` rows and `info` about any dataframe.  `info` in particular gives us very useful information about our data.  Pay close attention to the "non-null" count and the data type of each column.


In [3]:
print(iris.head(2))
print("")
print(iris.tail(2))
print("")
print(iris.info())

   sepal length  sepal width  petal length  petal width        class
0           5.1          3.5           1.4          0.2  Iris-setosa
1           4.9          3.0           1.4          0.2  Iris-setosa

     sepal length  sepal width  petal length  petal width           class
148           6.2          3.4           5.4          2.3  Iris-virginica
149           5.9          3.0           5.1          1.8  Iris-virginica

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal length  150 non-null    float64
 1   sepal width   149 non-null    float64
 2   petal length  150 non-null    float64
 3   petal width   150 non-null    float64
 4   class         150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB
None




The first issue that shows up is the missing value in "sepal width".  Every other column has 150 values, but this one has just 149.  Missing values in a table are called "null" values and they can mess with our analysis if we are not aware of them.  Sometimes we do want to leave them there, but often we want to exclude that data.

**YOU MUST NEVER MODIFY YOUR SOURCE DATA**

If there is "junk" in your input file, never make changes directly in the file, for the following reasons:
  * data is often audited and modification of official data could be an infringement
  * one person's junk is another person's treasure
  * pandas can adjust the data for you easily so you can have a "clean" version without adjusting the original

Lets start by finding that null value.  We know it is in the "sepal width" series


In [4]:
iris["sepal width"]

0      3.5
1      3.0
2      3.2
3      3.1
4      3.6
      ... 
145    3.0
146    2.5
147    3.0
148    3.4
149    3.0
Name: sepal width, Length: 150, dtype: float64

we can't see it in the preview, but pandas can filter a series to keep only the null values, but do that, we will need to understand indexing with masks.   

Firstly, we get the "sepal width" series from the data frame.  Then we call a function on that series which will convert all values to true or false.  If the original value was not null, we will get false, if it was null, we get true.  You will have to believe me, but there is a single "True" in there.

In [8]:
mask = iris["sepal width"].isnull()

print(mask.info())

mask

<class 'pandas.core.series.Series'>
RangeIndex: 150 entries, 0 to 149
Series name: sepal width
Non-Null Count  Dtype
--------------  -----
150 non-null    bool 
dtypes: bool(1)
memory usage: 278.0 bytes
None


0      False
1      False
2      False
3      False
4      False
       ...  
145    False
146    False
147    False
148    False
149    False
Name: sepal width, Length: 150, dtype: bool

We call a series that is all booleans a "mask" because of how we can use it in the next step.   You are used to putting a column name in the square brackets to get a series from a data frame but you can also put a mask in the square brackets.  If the mask has the same number of rows as the data frame, it will be used to choose which rows are given back.  Only indexes with a "True" in the mask result in a row being included in the result.

In [9]:
iris[mask]

,sepal length,sepal width,petal length,petal width,class
126,6.2,NaN,4.8,1.8,Iris-virginica


and thus we can see our problem.  There is a number missing in row 126.  Check the original dataframe to see for yourself.  Before you do, I will need to explain how to get a row from a data frame.  As well as `[]`, we have a method `iloc` which can take a row and column combination to get a value based on position.  _If you only give one position, the row with that index is selected_.

In [14]:
iris.loc[126]

sepal length               6.2
sepal width                NaN
petal length               4.8
petal width                1.8
class           Iris-virginica
Name: 126, dtype: object

ranges work as well though, so we can see the rows around the one we are interested in (Oh! Looks like there is another error in there, we will get to that soon).

In [15]:
iris.iloc[120:130]

,sepal length,sepal width,petal length,petal width,class
120,6.9,3.2,5.7,2.3,Iris-virginica
121,5.6,2.8,4.9,2.0,Iris-virginica
122,7.7,2.8,6.7,2.0,6.3
123,6.3,2.7,4.9,1.8,Iris-virginica
124,6.7,3.3,5.7,2.1,Iris-virginica
125,7.2,3.2,6.0,1.8,Iris-virginica
126,6.2,NaN,4.8,1.8,Iris-virginica
127,6.1,3.0,4.9,1.8,Iris-virginica
128,6.4,2.8,5.6,2.1,Iris-virginica
129,7.2,3.0,5.8,1.6,Iris-virginica


# Exercise

Identify all rows where the "petal width" is greater than 5.  How many are there?

In [20]:
print("put your answer in this code block")

(23, 5)


## Fixing the error

What do to about the error is up to you.  You should create a "clean" data frame, separate to the other one regardless of your decision.  Possible choices are:
  * remove that whole row
  * remove that whole column
  * choose a value for the missing entry

We will demonstrate each.


In [23]:
# remove that row

# we need the opposite mask. A trick to do this is to perform the "equals false" operation :)
mask2 = mask == False

# we can then use this as a mask to get only the rows we want.
clean_iris = iris[mask2]
clean_iris

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [24]:
# alternative!  Since we know exactly what row to drop, we can use the drop function

clean_iris = iris.drop(index=126)
clean_iris

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [27]:
# alternative - dropna does _all_ the hard work for us
clean_iris = iris.dropna()
clean_iris

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [26]:
# remove whole column with drop

clean_iris = iris.drop(columns="sepal width")
clean_iris

,sepal length,petal length,petal width,class
0,5.1,1.4,0.2,Iris-setosa
1,4.9,1.4,0.2,Iris-setosa
2,4.7,1.3,0.2,Iris-setosa
3,4.6,1.5,0.2,Iris-setosa
4,5.0,1.4,0.2,Iris-setosa
...,...,...,...,...
145,6.7,5.2,2.3,Iris-virginica
146,6.3,5.0,1.9,Iris-virginica
147,6.5,5.2,2.0,Iris-virginica
148,6.2,5.4,2.3,Iris-virginica


In [29]:
# choose a value for the missing entry with `fillna`
clean_iris = iris.fillna(0)
clean_iris.iloc[125:128]

,sepal length,sepal width,petal length,petal width,class
125,7.2,3.2,6.0,1.8,Iris-virginica
126,6.2,0.0,4.8,1.8,Iris-virginica
127,6.1,3.0,4.9,1.8,Iris-virginica


We used a couple of helper functions. [`dropna` will remove any rows with empty data](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html) while [`fillna` will replace any empty values with some other value](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html).

I tend not to fill in missing values, I tend to drop the whole row - that should be your default option but don't do it without paying attention to what is dropped and why.  Too many people just apply `dropna` without thinking.

In [ ]:
clean_iris = iris.dropna()

# Exercise

Identify the erroneous data in the `class` column and remove that row.

**Advanced** Imagine you had not seen the error, what type of pandas code could you construct to find such an error.  I would suggest extracting that column as a series, get all the unique values in the series (`drop_duplicates` will help).

In [31]:
print("put your solution here")

put your solution here


# Exercise

**Advanced** Find any other erroneous data and fix it.

In [ ]:
print("put your solution here")

# Concept Summary
  * We can index into a table with a mask
  * We can index into row with `iloc`
  * We can use boolean operators to create useful masks
  * `head`, `tail`, and `info` are useful for learning about your data
  * `dropna`, `drop_duplicates`, `duplicates`, `fillna` are usefull for cleaning your data

# Python concepts
  * `head`, `tail`, `info`, `dropna`, `drop_duplicates`, `duplicates`, `fillna` are all _methods_ on the data frame object
  * most of the methods we used returned entirely fresh values which we needed to capture in a new variable.  The original data frame was not changed by methods like `fillna`.
